In [4]:
#import library to connect to mysql database
import mysql.connector

In [5]:
#connect to umls database
umls_db = mysql.connector.connect(host='', port='',
                            user='', password='', database='')
print(umls_db)

mycursor = umls_db.cursor(buffered=True)

In [6]:
#function to interact with database
def fetch_result(sql):
    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    return myresult

In [7]:
#functions to write file
def write_to_file(data, file_name):
    f1 = open(file_name, "w")
    for i in range(len(data)):
        f1.write(str(data[i]) + '\n')
    f1.close()

In [11]:
def addEdge(g, node1, node2):
    if node1 not in g:
        g[node1] = []
    if node2 not in g:
        g[node2] = []
    g[node1].append(node2)

def populate_graph(cui_values, g):
    for item in cui_values:
        if item[0] is not item[1]:
            addEdge(g, item[0],item[1])

In [12]:
cycles=[]
def get_cycles(graph, node, visited, path):
    if len(cycles) < 20:
        visited[node] = True
        path.append(node)
        for neighbor in graph[node]:
            if neighbor in path:
                length = len(path) - path.index(neighbor)
                if length >= 3:
                    cycles.append(path[path.index(neighbor):])
            elif not visited[neighbor]:
                get_cycles(graph, neighbor, visited,path)
        path.pop()
        
def find_cycles(graph):
    visited = {node: False for node in graph}
    for node in graph:
         if not visited[node]:
            get_cycles(graph, node, visited, [])
    return cycles

In [13]:
# Write a SQL query to retrieve the data and relationships between the terms
sql = ("SELECT CUI1, CUI2 FROM MRREL WHERE REL='CHD'")
results = fetch_result(sql)
g={}

In [14]:
populate_graph(results, g)

In [15]:
cycles= find_cycles(g)

In [16]:
def get_disjoint_lists(lists):
    disjoint_lists = []
    for i in range(len(lists)):
        is_disjoint = True
        for j in range(i+1, len(lists)):
            if set(lists[i]) & set(lists[j]):
                is_disjoint = False
                break
        if is_disjoint:
            disjoint_lists.append(lists[i])
            if len(disjoint_lists) == 3:
                break
    return disjoint_lists

In [17]:
cycles= get_disjoint_lists(cycles)

In [18]:
print(cycles)

[['C0437720', 'C0437733', 'C0437737'], ['C0686705', 'C0161016', 'C0232070', 'C0238037'], ['C0042111', 'C0221013', 'C1541840']]


In [19]:
write_to_file(cycles, "output.txt")